In [5]:
import pandas as pd
import nltk
from datetime import datetime

In [67]:
data=pd.read_csv("data\\kred_unic_all.csv", encoding='cp1251',delimiter=';')
data['TEXT']

0         клиент в настоящий момент находится на отдыхе ...
1         ет договоров в работе, счет закрывать не будут...
2                                      плохое фин положение
3                   клиент попадает в просрочку. нет залога
4         клиент поставляет оборудование в сеть магазино...
5              кредит уже выдан, новый продукт не интересен
6          выдача кредита 2216/бизнес-оборот в декабре 2016
7         сфера деятельности клиента- арендный бизнес. и...
8                                                22.02.2017
9         достигнута договоренность о встрече на 08.02.2017
10                                       встреча 14 февраля
11        назначена встреча с клиентом на 21 февраля 201...
12                      на территории клиента в 16-00 09/02
13          назначена встреча с лпр на территории у клиента
14        руководитель и гл. бухгалтер отсутствуют. буде...
15        с клиентом проводили встречу в декабре прошлог...
16                                      

In [64]:
import datetime
import re

def valid_date(string):
    string=string.split(' ')
    parsed_string=''
    for token in datestring:
            try:
                 parsed_string+=' '+(re.sub(r'(\d{2}).(\d{2}).?(\d{0,4})', r'дата\время', token))
            except ValueError:
                 parsed_string+=' '+token
    return parsed_string

In [66]:
datestring=u'+7 (925) 545 встреча 18.03 подъеду в офис'
valid_date(datestring)


' +7 (925) 545 встреча дата\\время подъеду в офис'

In [69]:
nltk.span_tokenize(datestring)

AttributeError: module 'nltk' has no attribute 'span_tokenize'